In [ ]:
!pip install pyLDAvis

In [ ]:
!pip install spacy==2.2

In [ ]:
import os
os.kill(os.getpid(), 9)

## Data Cleaning

In [1]:
import os
import csv

In [2]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [3]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Important:

This Python notebook is designed to run in Google Colab. When you run the code below, an upload button will appear. Click the upload button, and choose the .csv file that you want to clean and analyze.

In [82]:
from google.colab import files 
  
uploaded = files.upload()

Saving telegram_privacy.csv to telegram_privacy.csv


In [6]:
import pandas as pd
import io

## Important:

Choose the python code below that match the name of the file that you uploaded above to read the csv file. For example, if you uploaded the telegram_2020.csv file, please use the df = pd.read_csv(io.BytesIO(uploaded['telegram_2020.csv'])) code and ignore the rest.

In [7]:
 
df = pd.read_csv(io.BytesIO(uploaded['telegram_2020.csv']))
df.head()

,text
0,#telegram paid service registrations are open ...
1,#Corona Info for #Germany:\r\nNew Cases: 9060\...
2,If you haven't checked out our #ICON #Telegram...
3,RT @africademics: Launch Alert: Join our new #...
4,RT @HannaLiubakova: #Belarus Day 120. Belarusi...


In [18]:
df = pd.read_csv(io.BytesIO(uploaded['telegram_2021.csv']))

In [29]:
df = pd.read_csv(io.BytesIO(uploaded['whatsapp_2020.csv']))

In [46]:
df = pd.read_csv(io.BytesIO(uploaded['whatsapp_2021.csv']))

In [83]:
df = pd.read_csv(io.BytesIO(uploaded['telegram_privacy.csv']))

In [74]:
df = pd.read_csv(io.BytesIO(uploaded['whatsapp_privacy.csv']))

In [55]:
df = pd.read_csv(io.BytesIO(uploaded['whatsapp_feature.csv']))

In [64]:
df = pd.read_csv(io.BytesIO(uploaded['telegram_feature.csv']))

In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['whatsapp.csv']))

In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['telegram.csv']))

## Important:

The dataset will be read as the df object. So all the codes below can be used, regardless of which dataset you uploaded.

In [84]:
X=df['text']
stop_words=stopwords.words('english')
stemmer=PorterStemmer()
import re
cleaned_data=[]
for i in range(len(X)):
    tweet=re.sub('[^a-zA-Z]',' ',X.iloc[i])
    tweet=tweet.lower().split()
    tweet=[stemmer.stem(word) for word in tweet if (word not in stop_words)]
    tweet=' '.join(tweet)
    cleaned_data.append(tweet)

In [85]:
df = pd.DataFrame(cleaned_data,columns=["tweets_cleaned"])
df.head()

,tweets_cleaned
0,rt itssssp peopl instal telegram whatsapp chan...
1,research mkm partner find facebook could face ...
2,rt dharmarajmc best choic whatsappnewpolici te...
3,latest privaci daili http co crm zgbxni whatsa...
4,ownyourprivaci share post start thedatatalk fa...


# LDA

In [86]:
import spacy
nlp = spacy.load("en_core_web_sm")
def lemmatizer(text):        
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)
new_tweets_df_clean = pd.DataFrame(df.tweets_cleaned.apply(lambda x: lemmatizer(x)))
new_tweets_df_clean['tweets_cleaned'] = new_tweets_df_clean['tweets_cleaned'].str.replace('-PRON-', '')

In [87]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(
analyzer='word',       
min_df=3,# minimum required occurences of a word 
stop_words='english',# remove stop words
lowercase=True,# convert all words to lowercase
token_pattern='[a-zA-Z0-9]{3,}',# num chars > 3
max_features=5000,# max number of unique words
                            )
data_matrix = vectorizer.fit_transform(new_tweets_df_clean.tweets_cleaned)
data_matrix

<394x413 sparse matrix of type '<class 'numpy.int64'>'
	with 3501 stored elements in Compressed Sparse Row format>

In [88]:
lda_model = LatentDirichletAllocation(
n_components=10, # Number of topics
learning_method='online',
random_state=20,       
n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_matrix)

In [89]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_matrix, vectorizer, mds='tsne')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:348: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  error = np.finfo(np.float).max
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
5      32.420952   8.115230       1        1  30.745612
3       0.648955  71.707100       2        1  23.851115
2      71.019684   9.971006       3        1  13.114678
4       3.785981  33.185135       4        1   6.961831
7     -42.666500  -9.222070       5        1   5.739108
9      -3.621689 -43.526516       6        1   5.700687
1      -3.632402  -4.171708       7        1   3.976993
0     -36.584400  35.804413       8        1   3.931314
8      44.391380  50.615383       9        1   3.517726
6      40.235809 -31.591665      10        1   2.460936, topic_info=         Term        Freq       Total Category  logprob  loglift
358  telegram  149.000000  149.000000  Default  30.0000  30.0000
20        amp   38.000000   38.000000  Default  29.0000  29.0000
183      http  328.000000  328.000000  Default  28.0000  28.0000
286   privaci  285.000000  285.000000  Default  27.0000  27.0000
278    polici   39.000000   39.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
275  platform    0.148675   10.542272  Topic10  -6.4525  -0.5568
31      becom    0.155334   14.685255  Topic10  -6.4087  -0.8444
325       say    0.142504    9.532357  Topic10  -6.4949  -0.4985
225      link    0.127904    6.790135  Topic10  -6.6030  -0.2673
183      http    0.179234  328.171358  Topic10  -6.2656  -3.8080

[420 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         2  0.721221     abandon
1         4  0.173180      accept
1         6  0.519540      accept
1         9  0.173180      accept
3         1  0.350846     account
...     ...       ...         ...
409       3  0.650153      youtub
410       2  0.577655       ysign
411       1  0.697200         zec
412       2  0.331651  zuckerberg
412       7  0.331651  zuckerberg

[601 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 3, 5, 8, 10, 2, 1, 9, 7])

In [90]:
for i,topic in enumerate(lda_model.components_):
  print(f'Top 10 words for topic #{i}:')
  print([vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
  print('\n')

Top 10 words for topic #0:
['fuw', 'readi', 'everi', 'locat', 'download', 'day', 'good', 'post', 'telegram', 'http']


Top 10 words for topic #1:
['becom', 'review', 'everyon', 'hear', 'nca', 'corrupt', 'encrochat', 'privaci', 'justic', 'amp']


Top 10 words for topic #2:
['regul', 'startup', 'threat', 'manag', 'complianc', 'gdpr', 'cybersecur', 'secur', 'privaci', 'http']


Top 10 words for topic #3:
['help', 'whatsapp', 'commun', 'right', 'infosec', 'new', 'secur', 'cybersecur', 'http', 'privaci']


Top 10 words for topic #4:
['servic', 'whatsapp', 'app', 'internet', 'agre', 'facebook', 'updat', 'term', 'polici', 'privaci']


Top 10 words for topic #5:
['app', 'late', 'signal', 'join', 'whatsappnewpolici', 'use', 'whatsapp', 'privaci', 'telegram', 'http']


Top 10 words for topic #6:
['telegram', 'sourc', 'coronatrackernep', 'bot', 'coronaviru', 'today', 'recoveri', 'covid', 'case', 'total']


Top 10 words for topic #7:
['datum', 'ransomwar', 'dataprotect', 'phish', 'malwar', 'privac

In [72]:
topic_values = lda_model.transform(data_matrix)
df['Topic'] = topic_values.argmax(axis=1)

In [16]:
df.head()

,tweets_cleaned,Topic
0,telegram paid servic registr open run calendar...,1
1,corona info germani new case today recoveri to...,4
2,check icon telegram bot pleas look step step g...,9
3,rt africadem launch alert join new scholarship...,6
4,rt hannaliubakova belaru day belarusian came s...,6


# VADER Sentiment Analyzer

In [17]:
sid = SentimentIntensityAnalyzer()
new_df = df['tweets_cleaned'].apply(lambda x: sid.polarity_scores(x))
new_df.head()

0    {'neg': 0.0, 'neu': 0.775, 'pos': 0.225, 'comp...
1    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
2    {'neg': 0.0, 'neu': 0.915, 'pos': 0.085, 'comp...
3    {'neg': 0.0, 'neu': 0.8, 'pos': 0.2, 'compound...
4    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
Name: tweets_cleaned, dtype: object

In [18]:
compound_sentiment = []
for i in range(0,len(new_df)):
    compound_sentiment.append(new_df.loc[i]['compound'])
print(compound_sentiment)

[0.8126, 0.0, 0.4019, 0.3612, 0.0, 0.5003, 0.5719, 0.0, 0.0, 0.0, 0.0, 0.0, 0.296, 0.4767, 0.0, 0.0, 0.0, 0.5423, 0.6124, 0.2732, 0.0, 0.0, 0.0, 0.8373, -0.296, 0.7717, 0.0, -0.128, 0.0, 0.0, 0.0, 0.872, 0.0, 0.0, 0.5859, 0.0, 0.4939, 0.2023, 0.0, 0.296, -0.5574, 0.3612, 0.6124, -0.5267, 0.0, 0.296, 0.0, -0.5423, 0.4019, 0.5106, 0.8591, 0.0, -0.2732, 0.0, 0.0, 0.296, -0.5267, -0.34, 0.0, 0.0, 0.0, 0.7845, 0.0, 0.296, -0.0258, 0.926, 0.1027, 0.8442, 0.6369, 0.0, -0.3182, 0.0, -0.25, 0.0, 0.0, -0.296, 0.0, 0.0, 0.0, -0.4767, -0.0772, 0.296, 0.0, 0.6249, 0.0, 0.0, 0.8316, 0.0, 0.0772, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.743, 0.5106, 0.0, 0.0, 0.0, 0.6249, 0.6597, 0.5719, -0.0772, 0.0, 0.0, 0.0516, 0.0, 0.5574, 0.0, 0.6369, 0.5106, 0.5106, 0.0, 0.0, 0.0, 0.3818, 0.0, -0.0772, 0.0, 0.0, 0.0, 0.0, 0.0, -0.34, 0.0, 0.0, 0.0, 0.0, 0.296, 0.5106, 0.0, 0.0, 0.0772, 0.0, -0.5719, 0.0, 0.0, 0.2732, 0.0, 0.0, -0.34, 0.0, 0.0, 0.0, -0.5267, 0.0, 0.0, 0.6369, -0.4939, -0.4472, -0.0772, 0.0, 0.0, 0.0, 0.6

In [19]:
df['compound_sentiment'] = pd.DataFrame(compound_sentiment)
df.head()

,tweets_cleaned,Topic,compound_sentiment
0,pleas help retweet dj bash open nice clubhous ...,9,0.8126
1,use watusi io fouadraheb add new featur whatsa...,8,0.0000
2,excit announc strateg partnership dreadbong dr...,2,0.4019
3,rt sarcastiqlonda user privaci data whatsapp l...,4,0.3612
4,whatsappnewpolici whatsapp whatsappprivacypoli...,0,0.0000


## Important:

Choose the python code below that match the name of the file that you uploaded above with _sentiment behind the name. For example, if you uploaded the telegram_2020.csv file, please use the df.to_csv('telegram_2020_sentiment.csv') code and ignore the rest. This code will export the dataframe with sentiment polarity score to a csv file.

In [58]:
df.to_csv('telegram_2020_sentiment.csv')
files.download('telegram_2020_sentiment.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [59]:
df.to_csv('telegram_2021_sentiment.csv')
files.download('telegram_2021_sentiment.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [76]:
df.to_csv('whatsapp_2020_sentiment.csv')
files.download('whatsapp_2020_sentiment.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
df.to_csv('whatsapp_2021_sentiment.csv')
files.download('whatsapp_2021_sentiment.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
df.to_csv('whatsapp_feature_sentiment.csv')
files.download('whatsapp_feature_sentiment.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
df.to_csv('whatsapp_privacy_sentiment.csv')
files.download('whatsapp_privacy_sentiment.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
df.to_csv('telegram_feature_sentiment.csv')
files.download('telegram_feature_sentiment.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
df.to_csv('telegram_privacy_sentiment.csv')
files.download('telegram_privacy_sentiment.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>